Detect motion with PIL based on the recipe for OpenCV in the reference. This is because installing OpenCV on Raspberry Pi is a bear.

In [103]:
from picamera import PiCamera, array
import numpy as np
from io import BytesIO
from PIL import Image, ImageFilter, ImageMorph, ImageEnhance
import time
import os

Detect motion by imaging in two takes, one after the other, and comparing pixels. Let's write a function to take a snap.

In [104]:
def take_motion_snap(width, height):
    with PiCamera() as Eye:
        time.sleep(1)
        Eye.resolution = (width, height)
        Eye.rotation = 180
        with array.PiRGBArray(camera=Eye) as Stream:
            Eye.exposure_mode = 'auto'
            Eye.awb_mode = 'auto'
            Eye.capture(Stream, format='rgb')
            return Stream.array

In [105]:
test = take_motion_snap(300, 300)
print("Got an image with width, height as {}, {}.".format(test.shape[0], test.shape[1]))
snap = Image.fromarray(test)
snap.show()

Got an image with width, height as 300, 300.


Now we have an RGB image of the specified height and width as a 3-dimensional numpy array. We want to take the difference between two snaps. Write a wrapper function to use ```take_motion_snap(w, h)```, take two snaps and return the computed difference.

In [106]:
def take_two_motion(intervalsec):
    im_one = take_motion_snap(300, 300)
    tic = time.time()
    toc = tic
    while (toc - tic) < intervalsec:
        toc = time.time()
    im_two = take_motion_snap(300, 300)
    im_diff = np.subtract(im_two, im_one)
    return im_diff


In [107]:
test_diff = take_two_motion(6)
print("Got a difference of width, height as {}, {}.".format(test_diff.shape[0], test_diff.shape[1]))
print("The median, mean diff are {:.2f}, {:.2f}.".format(np.median(test_diff), np.mean(test_diff)))
snap = Image.fromarray(test_diff)
snap.show()

Got a difference of width, height as 300, 300.
The median, mean diff are 23.00, 96.90.


Apply thresholding to remove noise from motion and nuisance effects such as lighting change. In thresholding, we will set the value of a pixel in each of the RGB channels to 0 (min) or 255 (max) accordng to a binary threshold value. In OpenCV, this operation would be ```cv2.threshold(frame_delta, 50, 255, cv2.THRESH_BINARY)```. Use numpy operations here.

In [108]:
def threshold_difference(imdiff, threshold=50):
    return np.uint8(np.where(imdiff > threshold, 255, 0))

In [109]:
diff_clean = threshold_difference(test_diff, np.mean(test_diff))
print("Got a thresholded image of width, height as {}, {}.".format(diff_clean.shape[0], diff_clean.shape[1]))
diff_clean.dtype
snap = Image.fromarray(diff_clean)
snap.show()

Got a thresholded image of width, height as 300, 300.


In [114]:
def erode_dilate(snap, showme=False):
    snap_eroded = snap.filter(ImageFilter.MinFilter(7))
    if showme:
        snap_eroded.show()
        print("After erosion, got median, mean as {:.2f}, {:.2f}.".format(np.median(snap_eroded), np.mean(snap_eroded)))
    snap_dilated = snap_eroded.filter(ImageFilter.MaxFilter(3))
    if showme:
        snap_dilated.show()
        print("After dilation, got median, mean as {:.2f}, {:.2f}.".format(np.median(snap_dilated), np.mean(snap_dilated)))
    snap_bnw = snap_dilated.convert('1')
    if showme:
        snap_bnw.show()
        print("B&W image for motion detection has dimensions {}".format(bnw_array.shape))
    bnw_array = np.array(snap_bnw)
    return bnw_array

In [115]:
test_bnw = erode_dilate(snap)
test_bnw

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [153]:
test_pass = np.zeros(test_bnw.shape, dtype = np.int32)  # Mask 
count_foreground = 0
first_pass_counter = 0
synonyms = {}

for idx,x in np.ndenumerate(test_bnw):
    aboveme = False
    leftofme = False
    if x: # Not background
        """
        Is there a pixel above or to the left that is not background?
        Check and if found, obtain the numeric labels 
        marking connected components in 1st pass.
        """
        count_foreground += 1
        if (idx[0] > 0): # Yes above
            aboveme = test_bnw[idx[0]-1, idx[1]]
            A = test_pass[idx[0]-1, idx[1]] # Get label
        if (idx[1] > 0): # Yes, on left
            leftofme = test_bnw[idx[0], idx[1]-1]
            B = test_pass[idx[0], idx[1]-1] # Get label
        """
        If both left and above have foreground,
        stick the lesser number as the label on our pixel.
        Note the conflict for 2nd pass correction.
        Note that if the lower value has already been marked
        for correction, follow the chain to the lowest value
        in the dictionary of synonymous labels.
        If only one of left or above have foreground,
        stick that label on our pixel.
        """
        if (aboveme and leftofme): # Contest           
            test_pass[idx] = min(A, B) # Resolve
            if (A != B): # Note for update in second pass
                if synonyms.get(min(A, B)):
                    synonyms[max(A, B)] = synonyms.get(min(A, B))
                else:
                    synonyms[max(A, B)] = min(A, B)
        elif aboveme:
            test_pass[idx] = A
        elif leftofme:
            test_pass[idx] = B        
        else:
            first_pass_counter += 1 # New label
            test_pass[idx] = first_pass_counter
            
print("Foreground has {} pixels.".format(count))
print("First pass found {} candidates.".format(first_pass_counter))
Image.fromarray(test_pass).show()
len(set(synonyms.values()))

Foreground has 20565 pixels.
First pass found 626 candidates.


51

In [150]:
for idx,x in np.ndenumerate(test_pass):
    if synonyms.get(x):
        test_pass[idx] = synonyms.get(x)
Image.fromarray(test_pass).show()
    

In [138]:
dict = {1: 0, 2: -1, -23: 4}
dict.get(-23)

4

Find connected regions.

## References:
1. A comprehensive DIY [guide](http://drsol.com/~deid/pi/camera/index.html) to Pi camera including many lesser-known techniques for image and video recording, processing and sharing.
2. A github [repo](https://gist.github.com/FutureSharks/ab4c22b719cdd894e3b7ffe1f5b8fd91) for pro motion detection with OpenCV.
3. A stackoverflow.com [post](https://stackoverflow.com/questions/31064974/whats-the-fastest-way-to-threshold-a-numpy-array) upon thresholding with operations upon numpy arrays.